In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

from py_vollib.black_scholes import black_scholes
from py_vollib.black_scholes.implied_volatility import implied_volatility

# Data Pre-processing


In [2]:
df = pd.read_csv("../res/fo27FEB2024bhav.csv", parse_dates=True).drop(
    columns=["Unnamed: 15"]
)
df

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP
0,FUTIDX,BANKNIFTY,29-Feb-2024,0.0,XX,46545.00,46745.00,46310.30,46587.20,46587.20,155766,1087604.62,1826475,-162900,27-FEB-2024
1,FUTIDX,BANKNIFTY,28-Mar-2024,0.0,XX,46706.00,47080.00,46670.20,46927.05,46927.05,73729,518627.57,1324575,257205,27-FEB-2024
2,FUTIDX,BANKNIFTY,25-Apr-2024,0.0,XX,47250.00,47415.00,47028.55,47279.60,47279.60,2966,21013.69,90945,7215,27-FEB-2024
3,FUTIDX,FINNIFTY,27-Feb-2024,0.0,XX,20590.00,20640.00,20500.15,20554.30,20552.90,9796,80620.26,234560,126680,27-FEB-2024
4,FUTIDX,FINNIFTY,26-Mar-2024,0.0,XX,20749.85,20759.05,20660.00,20682.00,20682.00,2331,19299.93,44960,26840,27-FEB-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55201,OPTSTK,ZYDUSLIFE,25-Apr-2024,980.0,PE,0.00,0.00,0.00,227.65,52.90,0,0.00,0,0,27-FEB-2024
55202,OPTSTK,ZYDUSLIFE,25-Apr-2024,1000.0,PE,0.00,0.00,0.00,246.85,66.10,0,0.00,0,0,27-FEB-2024
55203,OPTSTK,ZYDUSLIFE,25-Apr-2024,1020.0,PE,0.00,0.00,0.00,208.55,80.65,0,0.00,0,0,27-FEB-2024
55204,OPTSTK,ZYDUSLIFE,25-Apr-2024,1040.0,PE,0.00,0.00,0.00,285.70,96.40,0,0.00,0,0,27-FEB-2024


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55206 entries, 0 to 55205
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   INSTRUMENT  55206 non-null  object 
 1   SYMBOL      55206 non-null  object 
 2   EXPIRY_DT   55206 non-null  object 
 3   STRIKE_PR   55206 non-null  float64
 4   OPTION_TYP  55206 non-null  object 
 5   OPEN        55206 non-null  float64
 6   HIGH        55206 non-null  float64
 7   LOW         55206 non-null  float64
 8   CLOSE       55206 non-null  float64
 9   SETTLE_PR   55206 non-null  float64
 10  CONTRACTS   55206 non-null  int64  
 11  VAL_INLAKH  55206 non-null  float64
 12  OPEN_INT    55206 non-null  int64  
 13  CHG_IN_OI   55206 non-null  int64  
 14  TIMESTAMP   55206 non-null  object 
dtypes: float64(7), int64(3), object(5)
memory usage: 6.3+ MB


In [4]:
nse_calls = df.query(
    'SYMBOL == "NIFTY" & OPTION_TYP == "CE" & EXPIRY_DT == "29-Feb-2024"'
).drop(columns=["INSTRUMENT", "SYMBOL", "OPTION_TYP", "EXPIRY_DT", "TIMESTAMP"])

nse_calls

,STRIKE_PR,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI
4507,18650.0,3490.00,3548.70,3490.00,3541.65,3541.65,9,99.81,32400,-300
4508,18700.0,3430.00,3514.90,3400.00,3455.00,3505.25,89,986.39,51100,-4250
4509,18750.0,0.00,0.00,0.00,3076.00,3455.30,0,0.00,1150,0
4510,18800.0,3312.00,3360.00,3312.00,3336.05,3405.30,24,265.48,4400,-450
4511,18850.0,0.00,0.00,0.00,1710.25,3355.30,0,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...
4609,23750.0,0.60,1.05,0.50,0.55,0.55,26010,308877.52,243400,-27000
4610,23800.0,0.60,0.95,0.40,0.50,0.50,38342,456281.30,1038300,61250
4611,23850.0,0.50,0.80,0.35,0.40,0.40,19038,227032.94,653250,-98700
4612,23900.0,0.50,0.80,0.35,0.40,0.40,33367,398744.10,756400,-153350


In [5]:
S = 22193.8
nse_calls.loc[S - nse_calls["STRIKE_PR"] > 0, "MONEYNESS"] = "ITM"
nse_calls.loc[S - nse_calls["STRIKE_PR"] == 0, "MONEYNESS"] = "ATM"
nse_calls.loc[S - nse_calls["STRIKE_PR"] < 0, "MONEYNESS"] = "OTM"
nse_calls

,STRIKE_PR,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,MONEYNESS
4507,18650.0,3490.00,3548.70,3490.00,3541.65,3541.65,9,99.81,32400,-300,ITM
4508,18700.0,3430.00,3514.90,3400.00,3455.00,3505.25,89,986.39,51100,-4250,ITM
4509,18750.0,0.00,0.00,0.00,3076.00,3455.30,0,0.00,1150,0,ITM
4510,18800.0,3312.00,3360.00,3312.00,3336.05,3405.30,24,265.48,4400,-450,ITM
4511,18850.0,0.00,0.00,0.00,1710.25,3355.30,0,0.00,0,0,ITM
...,...,...,...,...,...,...,...,...,...,...,...
4609,23750.0,0.60,1.05,0.50,0.55,0.55,26010,308877.52,243400,-27000,OTM
4610,23800.0,0.60,0.95,0.40,0.50,0.50,38342,456281.30,1038300,61250,OTM
4611,23850.0,0.50,0.80,0.35,0.40,0.40,19038,227032.94,653250,-98700,OTM
4612,23900.0,0.50,0.80,0.35,0.40,0.40,33367,398744.10,756400,-153350,OTM


# Option Price Calculation


In [11]:
iv = lambda row: implied_volatility(
    row["SETTLE_PR"], S, row["STRIKE_PR"], 10 / 365, 0.01, "c"
)
nse_calls.apply(iv, axis=1)

BelowIntrinsicException: The volatility is below the intrinsic value.